In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from numpy import array
import pandas as pd
import cv2
from glob import glob
import PIL
import time
from tqdm import tqdm
import os
import seaborn as sns
import shutil
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array
import itertools
from tensorflow.keras.models import load_model

In [2]:
def patient_predict(patient_path, model_paths):
    def preprocess_image(img_path):
        img = load_img(img_path, color_mode='grayscale', target_size=(224, 224))
        img_array = img_to_array(img)
        pseudo_rgb_img = np.repeat(img_array, 3, axis=2)
        return preprocess_input(pseudo_rgb_img)

    def create_image_pairs(folder_path):
        images = []
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                processed_img = preprocess_image(img_path)
                images.append(processed_img)
        return list(itertools.combinations(images, 2)) if len(images) > 1 else [(images[0], images[0])]

    # Load models
    general_model = load_model(model_paths[0])
    specialized_model = load_model(model_paths[1])
    not_specialized_model = load_model(model_paths[2])

    image_pairs = create_image_pairs(patient_path)
    patient_results = []

    # Fixed weights
    weights = (0.4, 0.4, 0.2)

    for img1, img2 in image_pairs:
        img1 = np.expand_dims(img1, axis=0)
        img2 = np.expand_dims(img2, axis=0)
        combined_img = [img1, img2]

        general_pred = general_model.predict(combined_img)
        specialized_pred = specialized_model.predict(combined_img)
        not_specialized_pred = not_specialized_model.predict(combined_img)

        vote_sum = (general_pred * weights[0] +
                    specialized_pred * weights[1] +
                    not_specialized_pred * weights[2])
        weighted_vote = vote_sum >= sum(weights) / 2
        patient_results.append(int(weighted_vote))

    return "Abnormal" if np.max(patient_results) else "Normal"

# Usage
model_path = []
model_paths = ['mobilenet_combination_model_fold_all_data_3.h5', 'mobilenet_combination_model_fold_4bp_2.h5', 'mobilenet_combination_model_fold_all_exc_4_1.h5']
patient_path = '../datasets/test_patient'
predicted_label = patient_predict(patient_path, model_paths)
print("Predicted Label:", predicted_label)

1/1 [==============================] - 1s 927ms/step
Predicted Label: Normal


: 

In [20]:
import dill


# Serialize the function to a file
with open('patient_predict.pkl', 'wb') as f:
    dill.dump(patient_predict, f)

In [21]:
# Load the function from the file
with open('patient_predict.pkl', 'rb') as f:
    loaded_function = dill.load(f)

# Now you can use 'loaded_function' just like the original 'full_patient_analysis'
# Example usage:
model_paths = ['mobilenet_combination_model_fold_all_data_3.h5', 'mobilenet_combination_model_fold_4bp_2.h5', 'mobilenet_combination_model_fold_all_exc_4_1.h5']
patient_path = '../datasets/test_patient'
predicted_label = loaded_function(patient_path, model_paths)
print("Predicted Label:", predicted_label)

1/1 [==============================] - 0s 456ms/step
Predicted Label: Abnormal
